In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')
import seaborn as sns

# Read in Data

Latest Case and mobility data is from June 21, 2020

In [7]:
# import county level COVID-19 data
df_counties = pd.read_csv('Data/20200621_us-counties.csv')
df_counties['date_county'] = df_counties['date'] + ' ' + df_counties['county'] + ' ' + df_counties['state']

# import county level population density data
df_density = pd.read_csv('Data/county_density.csv').dropna()
df_density['GCT_STUB.display-label'] = df_density['GCT_STUB.display-label'].apply(lambda x: x.replace(' County', ''))
df_density['state_county'] = df_density['GEO.display-label'] + ' ' + df_density['GCT_STUB.display-label']

# import county level mobility data from Google's data reports
df_movement = pd.read_csv('Data/Global_Mobility_Report.csv')
df_movement = df_movement.drop(columns = ['iso_3166_2_code'])
df_movement = df_movement.loc[df_movement['country_region_code'] == 'US']
df_movement = df_movement.dropna().reset_index()
df_movement['sub_region_2'] = df_movement['sub_region_2'].apply(lambda x: x.replace(' County', ''))
df_movement['date_county'] = df_movement['date'] + ' ' + df_movement['sub_region_2'] + ' ' + df_movement['sub_region_1']
date_counties = np.array(df_counties['date_county'])

# match date county data accross mobility and COVID data 
df_movement = df_movement.loc[df_movement['date_county'].apply(lambda x: x in date_counties)]

df_merged = df_counties.join(df_movement.set_index('date_county'), how ='right', on='date_county', rsuffix='_other')

df_merged = df_merged.drop(['sub_region_1', 'sub_region_2', 'date_other', 'index', 'country_region_code'], axis=1).reset_index(drop=True)
df_merged['fips'] = df_merged['fips'].apply(lambda x: int(x))
df_merged['state_county'] = df_merged['state'] + ' ' + df_merged['county']

densities = []
for row in df_merged.itertuples():
    try:
        densities.append(df_density.loc[df_density['state_county'] == row.state_county]['Density'].values[0])
    except:
        densities.append(None)
df_merged['density'] = densities
df_merged['fips'] = df_merged['fips'].apply(lambda x: int(x))

# Parse date
df_merged['date'] = pd.to_datetime(df_merged['date'])
# Rename columns
df_merged = df_merged.rename(columns = {
    'retail_and_recreation_percent_change_from_baseline': 'retail',
    'grocery_and_pharmacy_percent_change_from_baseline' : 'grocery',
    'parks_percent_change_from_baseline': 'parks',
    'transit_stations_percent_change_from_baseline' : 'transit',
    'workplaces_percent_change_from_baseline' : 'workplace',
    'residential_percent_change_from_baseline': 'residential'
})

# Add weekdays and process dates
df_merged['weekday'] = df_merged['date'].map(lambda x: x.weekday())
# Get number of days since beginning of time period
start = df_merged['date'][0] # get first data
df_merged['days'] = df_merged['date'].map(lambda x: (x - start).days)

# Drop missing values
df_merged = df_merged.dropna()

/Users/ethankim/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


KeyError: 0

In [15]:
# Save processed data
df_merged.to_csv('Data/processed_mobility.csv')